In [37]:
import torch
import transformers
#from transformers import BertForQuestionAnswering
#from transformers import BertTokenizer
#MODIF
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
#FIN MODIF
import spacy
import en_core_web_sm
#MODIF
import fr_core_news_md
#FIN MODIF
import wikipedia
import wikipediaapi
from nltk.tokenize import word_tokenize

#MODIF (+ à installer)
#python -m spacy download fr_core_news_md
wikipedia.set_lang("fr")
from transformers import pipeline
from langdetect import detect
#FIN MODIF

# si besoin clone the repo
#git lfs install
#git clone https://huggingface.co/etalab-ia/camembert-base-squadFR-fquad-piaf

## Def class Ask / Load Model

In [38]:
class askfr:
    def __init__(self):
        #self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
        #self.model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
        #MODIF
        ##self.tokenizer = AutoTokenizer.from_pretrained("etalab-ia/camembert-base-squadFR-fquad-piaf")
        ##self.model = AutoModelForQuestionAnswering.from_pretrained("etalab-ia/camembert-base-squadFR-fquad-piaf")
        self.tokenizer = AutoTokenizer.from_pretrained("etalab-ia/camembert-base-squadFR-fquad-piaf")
        self.model = AutoModelForQuestionAnswering.from_pretrained("etalab-ia/camembert-base-squadFR-fquad-piaf")
        
        #FIN MODIF
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        #self.device = ('cpu')
        print(self.device)
        #move model to device
        self.model = self.model.to(self.device)
        #self.wiki = wikipediaapi.Wikipedia(language='en')
        #MODIF
        #self.nlp = fr_core_news_md.load()   #en_core_web_sm is better
        self.nlp = en_core_web_sm.load()
        self.wiki = wikipediaapi.Wikipedia(language='fr')
        self.nlppipeline = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')
        #FIN MODIF


   


    def generateAnswer(self, question, answer_text):
        # == Tokenize == Apply the tokenizer to the input text, treating them as a text-pair. (CPU)
        input_ids = self.tokenizer.encode(question, answer_text)
        tokens = self.tokenizer.convert_ids_to_tokens(input_ids)
        # == Set Segment IDs == Search the input_ids for the first instance of the `[SEP]` token.
        sep_index = input_ids.index(self.tokenizer.sep_token_id)
        # The number of segment A tokens includes the [SEP] token istelf.
        num_seg_a = sep_index + 1
        # The remainder are segment B.
        num_seg_b = len(input_ids) - num_seg_a
        # Construct the list of 0s and 1s.
        segment_ids = [0]*num_seg_a + [1]*num_seg_b
        # There should be a segment_id for every input token.
        assert len(segment_ids) == len(input_ids)
        # == Run Model == Run our example through the model. (GPU)
        #move tensor to device
        input_ids_tensor = torch.tensor([input_ids]).to(self.device)
        segment_ids_tensor = torch.tensor([segment_ids]).to(self.device)
    
    
        #print(input_ids_tensor)
        #print(input_ids_tensor.shape)
        #print(input_ids_tensor.size)
        #print("")
        #print(segment_ids_tensor)
        #print(segment_ids_tensor.shape)
        #print(segment_ids_tensor.size)
        
        start_scores, end_scores = self.model(input_ids_tensor, # The tokens representing our input text.
                                 token_type_ids=segment_ids_tensor) # The segment IDs to differentiate question from 
        # Find the tokens with the highest `start` and `end` scores.
        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        # get score
        start_score = float(start_scores[0,answer_start])
        end_score = float(end_scores[0,answer_end])
    
    
        # == Print Answer without ## ==
        # Start with the first token.
        answer = tokens[answer_start]

        # Select the remaining answer tokens and join them with whitespace.
        for i in range(answer_start + 1, answer_end + 1):
    
            # If it's a subword token, then recombine it with the previous token.
            if tokens[i][0:2] == '##':
                answer += tokens[i][2:]
    
            # Otherwise, add a space then the token.
            else:
                answer += ' ' + tokens[i]

        return answer, start_score+end_score

    def extract_subject_with_spacy(self, question):
    
        #question = truecase.get_true_case(question) #le truecaser est un peu bidon j'ai l'impression
        #print(question)
        
        subject_dict = {'subject' : '', 'infos' : []} #dictionnaire qui contiendra le sujet, et les infos complémentaires
        
        #osef_list = ['who','why','what','when','which','how', 'Who','Why','What','When','Which', 'How'] #noun to not take into account
        #MODIF
        osef_list = ['qui','pourquoi','quoi','quand','quel','quelle','comment','où','Qui','Pourquoi','Quoi','Quand','Quel','Quelle', 'Comment']
        #FIN MODIF
        doc = self.nlp(question)
        
        #on prépare une liste des noms communs (ou plus précisent chunks, qui peuvent être des groupes nominaux plus larges, des unités de sens) de la question
        
        nouns_list = []
        #dep_list = []
        for noun in doc.noun_chunks :
            #dep_list.append(noun.root.dep_)
            nouns_list.append(noun)
               
        #on enlève de la liste des potentiels sujets les mots interrogatifs venant de osef_list
        for noun in nouns_list : 
            if str(noun) in osef_list : 
                nouns_list.remove(noun)
                    
        #on crée une liste d'entité nommées de la phrase. S'il y en a une dans la question, alors c'est le sujet
        #nn crée également une liste qui va contenir les labels de ces entités nommées, car certains types d'entités ne nous intéressent pas
        #les labels qui nous intéressent sont dans la liste relevant_labels
        
        #si il y a des entités nommées, on les utilise comme sujet et les chunks alentours comme infos supplémentaires
        #si il n'y a pas d'entités nommées, on va uniquement regarder les chunks (dans le 'else')
        
        ents_list = []
        labels_list = []
        
        relevant_labels = ['PERSON','FAC','ORG','GPE','LOC','PRODUCT','EVENT','WORK_OF_ART','LAW']
        for ent in doc.ents : #TENTER SPACY FR 
            if ent.label_ in relevant_labels : 
                ents_list.append(ent.text)
                labels_list.append(ent.label_)
                #dep_list.append(ent.dep_) #pour tests sur depencies
                
        if ents_list and labels_list : 
            print(ents_list)
            #print(dep_list)
            print('subject found by ent : ', labels_list[-1] , ents_list[-1], '\n')
            subject_dict['subject'] = ents_list[-1] #on renvoie la dernière entité nommée pertinente trouvée
            for other_noun in nouns_list : 
                subject_dict['infos'].append(other_noun)
            return(subject_dict)
            
    
        else : 
            
    #si notre liste de chunks potentiels sujets est vide : pas de sujet
    #si elle est égal à 1 : pas de doute, le sujet est cet élément
    #si elle est plus grande que 1, le sujet est le deuxième élément
    #règle simpliste mais qui semble suivre la logique de la formulation d'une question : c'est souvent le second nom qui est le sujet dans les questions qui en comportent deux, j'ai l'impression 
        
            print(nouns_list)
            #print(dep_list)
            if(len(nouns_list)) == 0 :
                print("subject not found, please try another formulation", '\n')
            else :
                print("subject found by noun: " + str(nouns_list[-1]), '\n')
                subject_dict['subject'] = str(nouns_list[-1]) #le sujet est le dernier chunk
                for other_noun in nouns_list[0:-1] : #dans ces cas de figure avec + d'un nom, il faudra quand même récupérer le nom qui n'est pas le sujet, pour aller l'utiliser en scrappant la page wiki du sujet
                    subject_dict['infos'].append(other_noun)
                return(subject_dict)

    def get_sections_list(self, page):
        osef_list = ['Sources', 'Further reading', 'External links']
        def get_sections(sections, sections_list, level=0):
                for s in sections:
                        #print("%s: %s - %s" % ("*" * (level + 1), s.title, len(s.text)))
                        #check if there is text and if section is usefull
                        if len(s.text) != 0 and s.title not in osef_list:
                            sections_list.append(s.text)
                        get_sections(s.sections, sections_list, level + 1)
                        
        sections_list = []
        sections_list.append(page.summary)
        get_sections(page.sections, sections_list)
        return sections_list

    def get_paragraph(self, page):
        result = []
        result = self.get_sections_list(page)
        
        paragraph = []
        for section in result:
            for item in section.split("\n"):
                #check len <512 // 400-450
                if len(word_tokenize(item)) < 400:
                    paragraph.append(item)
        return paragraph

    def get_best_answer(self, question, subject):
        page = self.wiki.page(wikipedia.search(subject)[0])

        paragraph = self.get_paragraph(page)
        
        
        answers = []
        scores = []

        for p in paragraph[:15]:
            #answer, score = self.generateAnswer(question, p)
            #answers.append(answer)
            #scores.append(score)
            #MODIF
            nlpprocess = self.nlppipeline({'question': question,
                                          'context': p})
            answers.append(nlpprocess['answer'])
            print(nlpprocess['answer'])
            scores.append(nlpprocess['score'])
            print(nlpprocess['score'])
            #FIN MODIF
            
        max_value = max(scores)
        
        index = scores.index(max_value)
        return answers[index], paragraph[index], page.fullurl
        

    def run(self, question):
        sujet = self.extract_subject_with_spacy(question)
        a, p, u = self.get_best_answer(question, sujet['subject'])
        return a, p, u

In [39]:
askerfr = askfr()

cuda


### Example extraction subject modified

In [27]:
askerfr.extract_subject_with_spacy("qui est Nicolas Sarkozy ?")

['qui', 'Nicolas Sarkozy']
subject found by ent :  PERSON Nicolas Sarkozy 



{'subject': 'Nicolas Sarkozy', 'infos': []}

### Examples of the class method 

In [40]:
askerfr.run("qui est Nicolas Sarkozy ?")

['qui', 'Nicolas Sarkozy']
subject found by ent :  PERSON Nicolas Sarkozy 

Nicolas Sarközy de Nagy-Bocsa,
0.26079830527305603
président du conseil général des Hauts-de-Seine.
0.15897059440612793
président de la République
0.4885251224040985
Les Républicains.
0.10274647176265717
financement illicite de campagne électorale, recel de détournement de fonds publics libyens
0.06022872030735016
immigré hongrois, et d'Andrée Mallah
0.060354795306921005
fils d'un directeur général de la CIA,
0.41663965582847595
il est père de quatre enfants
0.8887252807617188
journaliste
0.3655945360660553
il entre à l'Institut d'études politiques de Paris
0.017394894734025
il appartient au Groupe rapide d'intervention,
0.041067760437726974
d'avocat d'affaires
0.5350726246833801
Arnaud Lagardère, qui le considère comme son « frère ».
0.4383656978607178
le fonds souverain du Qatar.
0.461905837059021
L'ancien chef de l'État,
0.6162910461425781


('il est père de quatre enfants',
 "Nicolas Sarkozy s'est marié trois fois et il est père de quatre enfants : Pierre (1985) et Jean (1986), nés de son mariage avec Marie-Dominique Culioli (mariés en 1982, divorcés en 1996), Louis (1997), né de son mariage avec Cécilia Ciganer-Albéniz (mariés en 1996, divorcés en 2007), et Giulia (2011), née de son mariage avec Carla Bruni-Tedeschi (le mariage a eu lieu le 2 février 2008 dans le Salon vert du palais de l'Élysée, sans publication des bans grâce à l'autorisation du procureur de la République, pour ne pas « troubler l'ordre public »). Son couple avec Cécilia fut largement médiatisé, y compris ses problèmes conjugaux en 2005-2007,,,.",
 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy')

In [9]:
askerfr.run("Où est né Napoléon Bonaparte ?")

[Napoléon Bonaparte]
subject found by noun: Napoléon Bonaparte 

Ajaccio
0.9691911935806274
en la cathédrale Notre-Dame de Paris,
0.25229862332344055
d'Italie au Nil
0.5392048954963684
La France
0.15998131036758423
Naples
0.9313117265701294
Portugal,
0.031144538894295692
Sainte-Hélène,
0.8043921589851379
Sainte-Hélène,
0.1900622844696045
Ajaccio
0.9015153050422668
d'Ajaccio
0.9577045440673828
avocat au Conseil supérieur de l'île
0.4956130087375641
15 août, un jour férié : la Saint-Napoléon.
0.17052310705184937
Ajaccio,
0.9792989492416382
milieu rural,
0.527215838432312
À l'école,
0.453364759683609


('Ajaccio,',
 "La famille Bonaparte vit à Ajaccio, rue Malerba (rue de la Mauvaise-Herbe, aujourd'hui rue Saint-Charles), dans une petite maison traditionnelle du XVIIIe siècle, que Napoléon qualifiera lui-même de « misérable ». La Casa Buonaparte est habitée au rez-de-chaussée et au premier étage par les Bonaparte et au deuxième étage par leurs cousins, les Pozzo di Borgo. Ce voisinage est insupportable et les deux familles vivent dans une brouille continuelle. On raconte qu'un jour, une Pozzo di Borgo aurait jeté le contenu d'un pot de chambre par la fenêtre, sur Madame Letizia.",
 'https://fr.wikipedia.org/wiki/Napol%C3%A9on_Ier')

In [18]:
askerfr.run("Qui est Mickey ?")

['Mickey']
subject found by ent :  ORG Mickey 



('Oswald le lapin chanceux,',
 "Mickey a été créé en 1928, après que Walt Disney eut dû laisser son premier personnage créé avec Ub Iwerks, Oswald le lapin chanceux, à son producteur. Les premiers courts métrages qui le mettent en scène sont principalement animés par Ub Iwerks, associé de Walt Disney au sein des studios Disney (alors installés dans le studio d'Hyperion Avenue). Il devient plus tard aussi un personnage de bandes dessinées, de longs métrages, de séries télévisées et d'une myriade de produits dérivés.",
 'https://fr.wikipedia.org/wiki/Mickey_Mouse')

In [32]:
#"IndexError: index out of range in self"
#"CamemBERT, like RoBERTa, does not make use of token type IDs."

#askerfr.generateAnswer("Où est né Napoléon Bonaparte ?","Napoléon Bonaparte, né le 15 août 1769 à Ajaccio et mort le 5 mai 1821 sur l'île Sainte-Hélène, est un militaire et homme d'État français, premier empereur des Français, du 18 mai 1804 au 6 avril 1814 et du 20 mars au 22 juin 1815, sous le nom de Napoléon Ier.")

### Examples of pipeline with very specified context (work very)

In [4]:
from transformers import pipeline

nlpp = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')

answers = []
scores = []

answer = nlpp({
    'question': "Qui est Claude Monet?",
    'context': "Claude Monet, né le 14 novembre 1840 à Paris et mort le 5 décembre 1926 à Giverny, est un peintre français et l’un des fondateurs de l'impressionnisme."
})

answer

0.5439729690551758

In [20]:
from transformers import pipeline

nlpp = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')

answers = []
scores = []

answer = nlpp({
    'question': "Quand risquons nous d'atteindre un réchauffement à 1.5 degrés?",
    'context': "Le réchauffement planétaire atteindra les 1,5 °C entre 2030 et 2052 si la température continue d'augmenter à ce rythme. Le RS15 (rapport spécial sur le réchauffement climatique de 1,5 °C) résume, d'une part, les recherches existantes sur l'impact qu'un réchauffement de 1,5 °C aurait sur la planète et, d'autre part, les mesures nécessaires pour limiter ce réchauffement planétaire."
})

answer

{'score': 0.7686043977737427,
 'start': 49,
 'end': 67,
 'answer': 'entre 2030 et 2052'}

In [19]:
from transformers import pipeline

nlpp = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')

answers = []
scores = []

answer = nlpp({
    'question': "Quelles recherches sont résumées dans ce rapport ?",
    'context': "Le réchauffement planétaire atteindra les 1,5 °C entre 2030 et 2052 si la température continue d'augmenter à ce rythme. Le RS15 (rapport spécial sur le réchauffement climatique de 1,5 °C) résume, d'une part, les recherches existantes sur l'impact qu'un réchauffement de 1,5 °C aurait sur la planète et, d'autre part, les mesures nécessaires pour limiter ce réchauffement planétaire."
})

answer

{'score': 0.002875491976737976,
 'start': 238,
 'end': 298,
 'answer': "l'impact qu'un réchauffement de 1,5 °C aurait sur la planète"}

In [33]:
#from transformers import pipeline
#
#nlp = pipeline('question-answering', model='camembert/camembert-large', tokenizer='camembert/camembert-large')
#
#answers = []
#scores = []
#
#answer = nlp({
#    'question': "Qui est Claude Monet?",
#    'context': "Claude Monet, né le 14 novembre 1840 à Paris et mort le 5 décembre 1926 à Giverny, est un peintre français et l’un des fondateurs de l'impressionnisme."
#})
#
#answer

#CAMEMBERT LARGE is not expected to do question answering task.

### Notes : Extraction ent for wikipedia with spacy

In [31]:
import spacy
nlp2 = spacy.load('fr_core_news_md')

question = "Où sont détaillées les Heures Creuses ?"

query = " ".join([str(x) for x in nlp2(question).ents])
query = query if len(query) > 0 else question

print(query)

relevant_title = wikipedia.search(query,results=10)

relevant_title

Heures Creuses


['Heure creuse',
 'Gare de Neuilly-Plaisance',
 'Chauffe-eau',
 'Ligne P du Transilien',
 'Ligne L du Transilien',
 'Gare de Bry-sur-Marne (RATP)',
 'Tesla Powerwall',
 'Ligne J du Transilien',
 "Ligne A du RER d'Île-de-France",
 'Ligne H du Transilien']

### Language detection with langdetect library

In [34]:
from langdetect import detect
detect("War doesn't show who's right, just who's left.")

'en'

In [35]:
detect("Ein, zwei, drei, vier")

'de'

In [36]:
detect("Où sont détaillées les Heures Creuses ?")

'fr'